In [2]:
import numpy as np
import pandas as pd
import math
import json
import sys
import time

In [1]:
# Reads a training set csv file and a restrictions vector text file, returns arranged training set          
def readFiles(filename=None, restrictions=None):
    if filename is None and restrictions is None:
        if len(sys.argv) < 2:
            print("Not enough arguments.")
            exit(1)
        elif len(sys.argv) == 3:
            restrictions = sys.argv[2]
        filename = sys.argv[1]

    restr=None
    if restrictions != None:
        with open(restrictions) as r:
            lines = r.read().replace(', ', ' ')
            restr = [int(x) for x in lines.split(' ')]

    df = pd.read_csv(filename)
    aclass = df.iloc[1,0]
    
    attrs = {}
    for a in df.columns:
        attrs[a] = int(df[a][0])
    
    isLabeled = True
    if not isinstance(aclass, str):
        isLabeled = False
    df = df.drop([0,1], axis=0)
    if restr != None:
        for i,v in enumerate(df.columns):
            if restr[i] == 0:
                df = df.drop(columns=[v])
    if isLabeled:
        df = df[[c for c in df if c not in [aclass]] + [aclass]]
        
    attrs.pop(df.columns[-1])
    return df, filename, isLabeled, attrs

In [5]:
# entropy of a series of data
def entropy(classcol):
    vals = classcol.value_counts()
    size = classcol.count()
    entropy=0
    for v in vals:
        entropy -= (v/size) * math.log(v/size,2)
    return entropy

# entropy of an attribute in a dataset, over each value of the attribute
def entropyAttr(data, attr):
    vals = data.pivot(columns=attr,values=data.columns[-1])
    entropyTot = 0
    for c in vals.columns:
        entropyTot += (vals[c].count()/len(data)) * entropy(vals[c])
    return entropyTot

# entropy of a series of data
def entropyValCounts(vals):
    size=0
    for v in vals:
        size += v
        
    entropy=0
    for v in vals:
        entropy -= (v/size) * math.log(v/size,2)
    return entropy

def splitEntropy(le, gt):
    sizeLe = 0
    for v in le:
        sizeLe += v
    sizeGt = 0
    
    for v in gt:
        sizeGt += v
    
    size = sizeLe+sizeGt
    
    return sizeLe/size * entropyValCounts(le) + sizeGt/size * entropyValCounts(gt)
    
def calcGainBetter(data,attr,p0):
    vals = data[attr].unique()
    
    bestSplit = None
    bestGain = 0.0
    for v in vals:
        le = data[data[attr] <= v].iloc[:,-1].value_counts()
        gt = data[data[attr] > v].iloc[:,-1].value_counts()
        
        splitGain = p0 - splitEntropy(le,gt)
        if splitGain > bestGain:
            bestGain = splitGain
            bestSplit = v
            
    return bestSplit, bestGain

def findBestSplit(data, attr, p0):
    return calcGainBetter(data, attr, p0)

In [ ]:
def selectSplittingAttr(attrs, data, threshold):
    p0 = entropy(data.iloc[:,-1])
    bestGain = 0
    alpha = None
    bestAttr = None
    
    for a in attrs:
        tmpAlpha=None
        tmpGain=0
        if attrs[a] < 1: # if attr is numeric
            tmpAlpha, tmpGain = findBestSplit(data, a, p0)
        else:
            tmpGain = p0 - entropyAttr(data, a)
        if tmpGain > bestGain:
            bestAttr = a
            bestGain = tmpGain
            alpha = tmpAlpha
    
    if bestGain > threshold:
        return bestAttr, alpha
    else:
        return None, None

In [51]:
def binarySplitEntropy(totalCounts, prevCounts):
    split2 = np.array(totalCounts) - np.array(prevCounts)
    size = np.sum(totalCounts)
    size1 = np.sum(prevCounts)
    size2 = np.sum(split2)
    
    entropy1=0
    for v in prevCounts:
        if v > 0:
            entropy1 -= (v/size1) * math.log(v/size1,2)
    
    entropy2=0
    for v in split2:
        if v > 0:
            entropy2 -= (v/size2) * math.log(v/size2,2)
    
    return (size1/size)*entropy1 + (size2/size)*entropy2

binarySplitEntropy()

1.584962500721156

In [73]:
def diskFindBestSplit(attr, data):
    p0 = entropy(data.iloc[:,-1])
    data = data.sort_values(by=attr)
    classes = data.iloc[:,-1].unique()
    k = len(classes)
    dataLen = len(data)
    
    alpha = [0] * dataLen
    counts = []
    for i in range(k):
        counts.append([0] * dataLen)
    
    l=0
    for _, v in data.iterrows():
        alpha[l] = float(v[attr])
        
        for j in range(k):
            if v[-1] == classes[j]: # if class of data point is jth class, -1 is last col
                counts[j][l] += 1
                
        l += 1
        
    totalCounts=[]
    for c in counts:
        totalCounts.append(np.sum(c))
    
    prevCounts=[0]*k
    l=0
    maxGain = 0
    bestAlpha = None
    for l in range(dataLen):
        for j in range(k):
            prevCounts[j] += counts[j][l] # sum previous counts
        gain = p0 - binarySplitEntropy(totalCounts, prevCounts)
        if gain > maxGain:
            maxGain = gain
            bestAlpha = alpha[l]
    return bestAlpha

df, filename, tmp, attrs = readFiles("./data/letter-recognition.data.csv")
diskFindBestSplit('xbox', df)

2.0

In [77]:
def calcGainBetter(data,attr):
    p0 = entropy(data.iloc[:,-1])
    vals = data[attr].unique()
    
    bestSplit = None
    bestGain = 0.0
    for v in vals:
        le = data[data[attr] <= v].iloc[:,-1].value_counts()
        gt = data[data[attr] > v].iloc[:,-1].value_counts()
        
        splitGain = p0 - splitEntropy(le,gt)
        if splitGain > bestGain:
            bestGain = splitGain
            bestSplit = v
            
    return bestSplit, bestGain

def findBestSplit(data, attr):
    return calcGainBetter(data, attr)

df, filename, tmp, attrs = readFiles("./data/letter-recognition.data.csv")
findBestSplit(df, 'xbox')

(2.0, 0.04797715771561428)

In [92]:
df, filename, tmp, attrs = readFiles("./data/crx.data.csv")
df[(df != '?').all(axis=1)]

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
2,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202,0.0,+
3,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43,560.0,+
4,a,24.5,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280,824.0,+
5,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100,3.0,+
6,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120,0.0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,b,21.08,10.085,y,p,e,h,1.25,f,f,0.0,f,g,260,0.0,-
688,a,22.67,0.750,u,g,c,v,2.00,f,t,2.0,t,g,200,394.0,-
689,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1.0,t,g,200,1.0,-
690,b,17.92,0.205,u,g,aa,v,0.04,f,f,0.0,f,g,280,750.0,-
